<a href="https://colab.research.google.com/github/karthik18-lgtm/Lip_sync_models/blob/main/Model_Orchestrator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# =========================================================================================
# MASTER SETUP SCRIPT (Wav2Lip + SadTalker in One Notebook)
# =========================================================================================

# --- Part 1: Initial Setup (Includes the asyncio patch) ---
print("--- Part 1: Initial Setup ---")
!pip install -q nest_asyncio
import nest_asyncio
nest_asyncio.apply()
%cd /content

# --- Part 2: Set up Wav2Lip ---
print("\n--- Part 2: Setting up Wav2Lip ---")
!git clone -q https://github.com/Rudrabha/Wav2Lip.git
!wget -q 'https://www.adrianbulat.com/downloads/python-fan/s3fd-619a316812.pth' -O /content/Wav2Lip/face_detection/detection/sfd/sfd.pth
!wget -q 'https://iiitaphyd-my.sharepoint.com/personal/radrabha_m_research_iiit_ac_in/_layouts/15/download.aspx?share=EAbPAoQh2kiMssg2152Q2XgBogImrcVQ0coD-2YW91Lh8Q' -O /content/Wav2Lip/checkpoints/wav2lip_gan.pth

# --- Part 3: Set up SadTalker ---
print("\n--- Part 3: Setting up SadTalker ---")
!git clone -q https://github.com/OpenTalker/SadTalker.git
%cd /content/SadTalker
!bash scripts/download_models.sh
%cd /content

# --- Part 4: Install All Combined Dependencies ---
print("\n--- Part 4: Installing all combined dependencies ---")
!pip install -q torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
!pip install -q librosa==0.9.2 basicsr gfpgan gradio dlib-bin face-alignment kornia yacs uvicorn==0.19.0 opencv-python imageio imageio-ffmpeg PyYAML tqdm scipy safetensors

# --- Part 5: Apply All Code Patches ---
print("\n--- Part 5: Applying all code patches ---")
!sed -i "s/from torchvision.transforms.functional_tensor import rgb_to_grayscale/from torchvision.transforms.functional import rgb_to_grayscale/g" /usr/local/lib/python3.12/dist-packages/basicsr/data/degradations.py
!sed -i "s/np.VisibleDeprecationWarning/DeprecationWarning/g" /content/SadTalker/src/face3d/util/preprocess.py
!sed -i "s/np.float/float/g" /content/SadTalker/src/face3d/util/my_awing_arch.py
!sed -i '101i \    t = t.flatten()' /content/SadTalker/src/face3d/util/preprocess.py

print("\n\n✅✅✅ Master setup of both models is complete. You can now run the application code. ✅✅✅")

--- Part 1: Initial Setup ---
/content

--- Part 2: Setting up Wav2Lip ---

--- Part 3: Setting up SadTalker ---
/content/SadTalker
--2025-10-22 05:06:28--  https://github.com/OpenTalker/SadTalker/releases/download/v0.0.2-rc/mapping_00109-model.pth.tar
Resolving github.com (github.com)... 20.205.243.166
Connecting to github.com (github.com)|20.205.243.166|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://release-assets.githubusercontent.com/github-production-release-asset/569518584/ccc415aa-c6f4-47ee-8250-b10bf440ba62?sp=r&sv=2018-11-09&sr=b&spr=https&se=2025-10-22T06%3A01%3A58Z&rscd=attachment%3B+filename%3Dmapping_00109-model.pth.tar&rsct=application%2Foctet-stream&skoid=96c2d410-5711-43a1-aedd-ab1947aa7ab0&sktid=398a6654-997b-47e9-b12b-9515b896b4de&skt=2025-10-22T05%3A01%3A04Z&ske=2025-10-22T06%3A01%3A58Z&sks=b&skv=2018-11-09&sig=ib9Di%2FLoS7ZqJJzRRvXF1P6JuJWePVj26Z3WabnAm28%3D&jwt=eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJpc3MiOiJnaXRodWIuY29tIiwi

In [2]:
import gradio as gr
import subprocess
import os
import glob

# --- Final Orchestrator Function ---

def run_lipsync_model(source_image, driving_audio, model_name):
    if source_image is None or driving_audio is None: return None
    print(f"--- Running model: {model_name} ---")

    # --- ROUTE TO WAV2LIP ---
    if model_name == "Wav2Lip":
        output_path = "/content/Wav2Lip/results/result_voice.mp4"
        command = ['python', 'inference.py', '--checkpoint_path', 'checkpoints/wav2lip_gan.pth', '--face', source_image, '--audio', driving_audio]
        try:
            print("Executing Wav2Lip...")
            subprocess.run(command, check=True, cwd="/content/Wav2Lip", capture_output=True, text=True)
            print("Wav2Lip finished.")
            return output_path
        except (subprocess.CalledProcessError, FileNotFoundError) as e:
            print("Wav2Lip failed:", e.stderr if hasattr(e, 'stderr') else "Folder not found.")
            return None

    # --- ROUTE TO SADTALKER ---
    elif "SadTalker" in model_name:
        command = ['python', 'inference.py', '--driven_audio', driving_audio, '--source_image', source_image, '--result_dir', './results', '--enhancer', 'gfpgan']
        if model_name == "SadTalker (Still Mode)": command.append('--still')

        try:
            print(f"Executing {model_name}...")
            subprocess.run(command, check=True, cwd="/content/SadTalker", capture_output=True, text=True)
            print("SadTalker finished.")
            list_of_files = glob.glob('/content/SadTalker/results/*.mp4')
            if not list_of_files: return None
            return max(list_of_files, key=os.path.getctime)
        except (subprocess.CalledProcessError, FileNotFoundError) as e:
            print("SadTalker failed:", e.stderr if hasattr(e, 'stderr') else "Folder not found.")
            return None
    return None

# --- Gradio Interface Construction ---
with gr.Blocks(title="Lip-Sync Application") as demo:
    gr.Markdown("# Lip-Sync Model Application\nSelect a model, upload an image and audio, and generate a video.")
    with gr.Row():
        image_input = gr.Image(label="Source Image", type="filepath")
        audio_input = gr.Audio(label="Driving Audio", type="filepath")
    model_selector = gr.Dropdown(choices=["Wav2Lip", "SadTalker", "SadTalker (Still Mode)"], value="SadTalker", label="Select Model")
    submit_btn = gr.Button("Generate Video", variant="primary")
    video_output = gr.Video(label="Resulting Video")
    submit_btn.click(fn=run_lipsync_model, inputs=[image_input, audio_input, model_selector], outputs=[video_output])

print("Launching the final application...")
demo.launch(share=True, debug=True)

Launching the final application...
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://867bfe6dcd5af7e7d4.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


--- Running model: SadTalker (Still Mode) ---
Executing SadTalker (Still Mode)...
SadTalker finished.
Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://867bfe6dcd5af7e7d4.gradio.live
